In [1]:
!pip uninstall tensorflow

Uninstalling tensorflow-2.5.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.5.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.5.0


In [2]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 35kB/s 
     |████████████████████████████████| 3.8MB 53.0MB/s 
     |████████████████████████████████| 512kB 45.2MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=b4828d61e568d32c6646f9b652e953f48ea6efab737c3dc6fce8905061db7ad0
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.0 which is incompatible.
  Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Found existing installati

In [3]:
!pip uninstall scipy

Uninstalling scipy-1.4.1:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/scipy-1.4.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/scipy/*
Proceed (y/n)? y
  Successfully uninstalled scipy-1.4.1


In [4]:
!pip install scipy==1.1.0

     |████████████████████████████████| 31.2MB 165kB/s 
ERROR: pymc3 3.11.2 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [6]:
import tensorflow as tf

In [7]:
tf.__version__

'1.15.0'

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
from __future__ import print_function
from __future__ import absolute_import

import tensorflow as tf
import argparse
import os

from unet import UNet

num = 1
experiment_dir = './gdrive/MyDrive/englishwords/' + str(num) + '/experiment/'
if not os.path.isdir(experiment_dir):
  os.mkdir(experiment_dir)

experiment_id = 0
image_size = 256

L1_penalty, Lconst_penalty, Ltv_penalty, Lcategory_penalty = 100, 15, 0.0, 1.0
embedding_num, embedding_dim = 1, 866

epoch = 50
batch_size = 16
lr = 0.0005
sample_steps, checkpoint_steps = 200, 500

schedule = 10
resume = 1
freeze_encoder = 0
fine_tune = None
inst_norm = 0
flip_labels = None


config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:
    model = UNet(experiment_dir, batch_size=batch_size, experiment_id=experiment_id,
                    input_width=image_size, output_width=image_size, embedding_num=embedding_num,
                     embedding_dim=embedding_dim, L1_penalty=L1_penalty, Lconst_penalty=Lconst_penalty,
                     Ltv_penalty=Ltv_penalty, Lcategory_penalty=Lcategory_penalty)
    model.register_session(sess)
    if flip_labels:
        model.build_model(is_training=True, inst_norm=inst_norm, no_target_source=True)
    else:
        model.build_model(is_training=True, inst_norm=inst_norm)
    fine_tune_list = None
    if fine_tune:
        ids = fine_tune.split(",")
        fine_tune_list = set([int(i) for i in ids])
    model.train(lr=lr, epoch=epoch, resume=resume,
                    schedule=schedule, freeze_encoder=freeze_encoder, fine_tune=fine_tune_list,
                    sample_steps=sample_steps, checkpoint_steps=checkpoint_steps,
                    flip_labels=flip_labels)






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





unpickled total 780 examples
unpickled total 86 examples
train examples -> 780, val examples -> 86


INFO:tensorflow:Restoring parameters from ./gdrive/MyDrive/englishwords/19/experiment/checkpoint/experiment_0_batch_16/unet.model-2450
restored model ./gdrive/MyDrive/englishwords/19/experiment/checkpoint/experiment_0_batch_16
Epoch: [ 0], [   0/  49] time: 18.7803, d_loss: 0.67654, g_loss: 12.52177, category_loss

In [ ]:
from __future__ import print_function
from __future__ import absolute_import

import tensorflow as tf
import os
import argparse
from unet import UNet
from utils import compile_frames_to_gif

num = 1
model_dir = './gdrive/MyDrive/englishwords/' + str(num) + '/experiment/checkpoint/experiment_0_batch_16/'
source_obj = './gdrive/MyDrive/englishwords/' + str(num) + '/experiment/data/test.obj'
save_dir = './'
if not os.path.isdir(save_dir):
  os.mkdir(save_dir)

batch_size = 1

embedding_num, embedding_dim = 1, 866
embedding_ids = '0'

inst_norm = 0
steps = 10
output_gif = 'test.gif'
uroboros = 0

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:
    model = UNet(batch_size=batch_size, embedding_num=embedding_num, embedding_dim=embedding_dim)
    model.register_session(sess)
    model.build_model(is_training=False, inst_norm=inst_norm)
    embedding_ids = [int(i) for i in embedding_ids.split(",")]
    embedding_ids = embedding_ids[0]
    model.infer(model_dir=model_dir, source_obj=source_obj, embedding_ids=embedding_ids, save_dir=save_dir)





The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


unpickled total 5 examples
examples -> 5



INFO:tensorflow:Restoring parameters from ./gdrive/MyDrive/englishwords/15/experiment/checkpoint/experiment_0_batch_16/unet.model-6000
restored model ./gdrive/MyDrive/englishwords/15/experiment/checkpoint/experiment_0_batch_16/
generated images saved at ./inferred_0005.png
